# Paramètres globaux, initialisation

In [1]:
nom_bandes = ['400', '412', '442', '490', '510', '560', '620', '665', 
              '674', '681', '709', '754', '761', '764', '768', '779', 
              '865', '885', '900', '940', '1020']
ind_bandes = '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21'

runLUTtool  = '/home/jovyan/bin/run6SV.ksh'
REP_in6SV   = 'NotUsed'
satellite   = 'SENTINEL3B'
capteur     = 'OLCIB'
bande       = '400' #Il est possible aussi d'itérer sur toutes les bandes
AER_NOM     = 'DESERT'
AER_TYPE    = '5'
NOMFICBANDE = 'Bande_%s_%s_%s.dat' % (satellite, capteur, bande)
REP_out6SV  = 'NotUsed'
REP_CODE_TR = 'NotUsed'
REP_WORK    = 'NotUsed'
ficJobArray = 'NotUsed'

FIC_CAPTEUR='/home/jovyan/aux/rep6s_SENTINEL3B_OLCIB.dat'
FICALLBANDS=FIC_CAPTEUR

In [2]:
import pandas as pd
def generate_fic_band_6sV(fic_capteur, band_name, fic_out, nom_bandes):
    all_bands_df = pd.read_csv(fic_capteur, delim_whitespace=True, names=['index'] + nom_bandes)
    #On ne garde que la bande qui nous intéresse
    band_df = all_bands_df.loc[:, ['index', band_name]]
    #On ne garde que les valeur supérieurs à 0
    band_df = band_df[band_df[band_name] > 0]

    # Génération au format 6S : 1ere ligne longueur d'onde début et fin
    # lignes suivantes, valeurs pour la bande donnée
    index_values = list(band_df['index'])
    band_values = list(band_df.iloc[:,1])
    with open(fic_out, 'w') as f:
        f.write("%.7E  %.7E\n" % (index_values[0], index_values[-1]))
        for value in band_values:
            f.write("%.7E\n" % value)

In [11]:
# Génération du fichier bande
generate_fic_band_6sV(FIC_CAPTEUR, bande, ('/home/jovyan/Bande_SENTINEL3B_OLCIB_%s.dat' % bande), nom_bandes)

# Les paramètres de simu

In [4]:
#### Parametres de la LUT
fields = ('u03', 'uw', 'ep_opt', 'psurf', 'alti', 'phi', 'thetas', 'thetav')

### A remplir





In [5]:
%%time
#Création des paramètres de simulation
lines = []
for u03 in tab_uO3:
    for uw in tab_uw:
        for ep_opt in tab_ep_opt:
            for psurf in tab_psurf:
                for alti in tab_alti:
                    for phi in tab_phi:
                        for thetas in tab_thetas:
                            for thetav in tab_thetav:
                                lines += [' '.join((runLUTtool, REP_in6SV, satellite, capteur, bande, AER_NOM, AER_TYPE, thetav, phi, thetas, ep_opt, uw, u03, psurf, alti, NOMFICBANDE, REP_out6SV, REP_CODE_TR, REP_WORK))]


CPU times: user 2.3 s, sys: 526 ms, total: 2.82 s
Wall time: 2.82 s


In [15]:
print(len(lines))
print(lines[10])

3123120
/home/jovyan/bin/run6SV.ksh NotUsed SENTINEL3B OLCIB 400 DESERT 5 50 0 0 0.1 0.0 0.20 940.0 0.100 Bande_SENTINEL3B_OLCIB_400.dat NotUsed NotUsed NotUsed


In [18]:
# Exécution d'un appel à sisv
import os
from subprocess import Popen, PIPE
def run_6v(cmd):
    process = Popen(cmd.split(' '), stdout=PIPE, cwd='/home/jovyan/bin')
    output, err = process.communicate()
    exit_code = process.wait()
    if exit_code != 0:
        raise ValueError('Echec de la commande ' + cmd)
    return output

In [20]:
run_6v(lines[15])

b'10 5 0 0.100 940.0 0.1 0.0 0.20\n'

# Dask sur Kubernetes

In [21]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.scale(4)
cluster

In [22]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.48.13.12:42379 Dashboard: /user/guillaumeeb-demo-lut-cloud-nh99y5g4/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Test de soumission

In [23]:
future = client.submit(run_6v, lines[15])
result = client.gather(future)
result

b'10 5 0 0.100 940.0 0.1 0.0 0.20\n'

In [24]:
%%time
futures = client.map(run_6v, lines[:1000])
results = client.gather(futures)

CPU times: user 10.9 s, sys: 1.25 s, total: 12.2 s
Wall time: 44.5 s


### Mode adaptatif

In [25]:
cluster.adapt(minimum=2, maximum=40)

In [26]:
futures = client.map(run_6v, lines[:10000])
results = client.gather(futures)

In [27]:
futures = client.map(run_6v, lines[:10200])
results = client.gather(futures)

In [28]:
results[0]

b'0 0 0 0.100 940.0 0.1 0.0 0.20\n'

In [29]:
results[9999]

b'10 10 180 0.700 990.0 0.1 0.0 0.20\n'

In [30]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
